In [3]:
import re
import os

In [1]:
import pandas as pd
import numpy as np

In [2]:
from bs4 import BeautifulSoup             

In [29]:
from nltk.corpus import stopwords
import nltk.data

In [75]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.cluster import KMeans

In [42]:
import gensim
from gensim.models import word2vec

In [39]:
import logging

-----
# Part 1: For Beginners - Bag of Words

[source](https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-1-for-beginners-bag-of-words)

## Reading the Data

In [6]:
train = pd.read_table('data/labeledTrainData.tsv', 
                      delimiter = '\t', 
                      quoting = 3)

train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [21]:
test = pd.read_table('data/testData.tsv', 
                     delimiter = '\t', 
                     quoting = 3)

test.head()

,id,review
0,"""12311_10""","""Naturally in a film who's main themes are of ..."
1,"""8348_2""","""This movie is a disaster within a disaster fi..."
2,"""5828_4""","""All in all, this is a movie for kids. We saw ..."
3,"""7186_2""","""Afraid of the Dark left me with the impressio..."
4,"""12128_7""","""A very accurate depiction of small time mob l..."


In [99]:
unlabeled_train = pd.read_csv("data/unlabeledTrainData.tsv", 
                              delimiter = "\t", 
                              quoting = 3)

unlabeled_train.head()

,id,review
0,"""9999_0""","""Watching Time Chasers, it obvious that it was..."
1,"""45057_0""","""I saw this film about 20 years ago and rememb..."
2,"""15561_0""","""Minor Spoilers<br /><br />In New York, Joan B..."
3,"""7161_0""","""I went to see this film with a great deal of ..."
4,"""43971_0""","""Yes, I agree with everyone on this site this ..."


------
## Data Cleaning and Text Preprocessing

In [7]:
def review_to_words(raw_review):
    """Function to convert a raw review to a string of words
    The input is a single string (a raw movie review), and 
    the output is a single string (a preprocessed movie review)
    """

    review_text = BeautifulSoup(raw_review, 'lxml').get_text() 
    letters_only = re.sub("[^a-zA-Z]", " ", review_text) 
    words = letters_only.lower().split()                             
    
    stops = set(stopwords.words("english"))                  
    meaningful_words = [w for w in words if not w in stops]   

    return(" ".join( meaningful_words ))   

# review_to_words( train["review"][0] )

In [8]:
%%time
clean_train_reviews = [review_to_words(review) for review in train.review]

CPU times: user 26.4 s, sys: 1.76 s, total: 28.2 s
Wall time: 29.7 s


------
## Creating Features from a Bag of Words (Using `scikit-learn`)

In [9]:
vectorizer = CountVectorizer(analyzer = "word",
                             tokenizer = None,
                             preprocessor = None,
                             stop_words = None,
                             max_features = 5000) 

In [10]:
%%time
train_data_features = vectorizer.fit_transform(clean_train_reviews)

CPU times: user 4.09 s, sys: 117 ms, total: 4.2 s
Wall time: 4.22 s


In [11]:
train_data_features = train_data_features.toarray()
# print(train_data_features.shape)

In [14]:
vocab = vectorizer.get_feature_names()
# print(vocab)

If you're interested, you can also print the counts of each word in the vocabulary:

In [17]:
# # Sum up the counts of each vocabulary word
# dist = np.sum(train_data_features, axis=0)

# # For each, print the vocabulary word and the number of times it 
# # appears in the training set
# for tag, count in zip(vocab, dist):
#     print(count, tag)

----
## Random Forest

Initializing a Random Forest classifier with 100 trees and fitting the forest to the training set, using the bag of words as features and the sentiment labels as the response variable.

In [18]:
forest = RandomForestClassifier(n_estimators = 100) 

In [19]:
%%time
forest = forest.fit(train_data_features, train["sentiment"])

CPU times: user 2min 10s, sys: 4.06 s, total: 2min 14s
Wall time: 2min 22s


----
## Making Predictions

Doing the same stuff, but with the test data:

In [22]:
%%time
clean_test_reviews = [review_to_words(review) for review in test.review]

CPU times: user 26.5 s, sys: 1.94 s, total: 28.4 s
Wall time: 30.1 s


In [23]:
%%time
test_data_features = vectorizer.transform(clean_test_reviews)

CPU times: user 4.78 s, sys: 225 ms, total: 5 s
Wall time: 5.29 s


In [24]:
test_data_features = test_data_features.toarray()

In [25]:
%%time
result = forest.predict(test_data_features)

CPU times: user 2.87 s, sys: 1.73 s, total: 4.61 s
Wall time: 6.94 s


In [ ]:
# # Creating a Submission
# output = pd.DataFrame( data={"id":test["id"], "sentiment":result} )
# output.to_csv("submissions/Bag_of_Words_model.csv", 
#               index = False, 
#               quoting = 3)

-------
# Part 2: Word Vectors

[source](https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-2-word-vectors)

In [52]:
def review_to_wordlist(review, remove_stopwords = False):
    """Function to convert a document to a sequence of words,
    optionally removing stop words.
    Returns a list of words.
    """
    review_text = BeautifulSoup(review, 'lxml').get_text()
    review_text = re.sub("[^a-zA-Z]"," ", review_text)
    words = review_text.lower().split()
    
    if remove_stopwords:
        stops = set(stopwords.words("english"))
        words = [w for w in words if not w in stops]
    
    return(words)

Load the `punkt` tokenizer

In [31]:
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

In [53]:
def review_to_sentences(review, tokenizer, remove_stopwords = False):
    """Function to split a review into parsed sentences.
    Returns a list of sentences, 
    where each sentence is a list of words
    """
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    
    # 2. Loop over each sentence
    sentences = []
    
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append(review_to_wordlist(raw_sentence, remove_stopwords))
    
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [55]:
%%time
sentences = []

for review in train["review"]:
    sentences += review_to_sentences(review, tokenizer)

/usr/local/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
/usr/local/lib/python3.5/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup


CPU times: user 2min 27s, sys: 8.11 s, total: 2min 35s
Wall time: 2min 46s


In [56]:
%%time
for review in unlabeled_train["review"]:
    sentences += review_to_sentences(review, tokenizer)

/usr/local/lib/python3.5/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You shouldprobably open this file and pass the filehandle intoBeautiful Soup.
  'Beautiful Soup.' % markup)
/usr/local/lib/python3.5/site-packages/bs4/__init__.py:282: UserWarning: "http://www.archive.org/details/LovefromaStranger"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.5/site-packages/bs4/__init__.py:282: UserWarning: "http://www.loosechangeguide.com/LooseChangeGuide.html"" looks like a URL. Beautiful Soup is not an HTTP client. You should probably use an HTTP client like requests to get the document behind the URL, and feed that document to Beautiful Soup.
  ' that document to Beautiful Soup.' % decoded_markup
/usr/local/lib/python3.5/site-packages

CPU times: user 4min 42s, sys: 20.1 s, total: 5min 2s
Wall time: 5min 18s


In [37]:
print(len(sentences))

795538


In [38]:
print(sentences[0])

['with', 'all', 'this', 'stuff', 'going', 'down', 'at', 'the', 'moment', 'with', 'mj', 'i', 've', 'started', 'listening', 'to', 'his', 'music', 'watching', 'the', 'odd', 'documentary', 'here', 'and', 'there', 'watched', 'the', 'wiz', 'and', 'watched', 'moonwalker', 'again']


-----
## Training and Saving Your Model

In [40]:
logging.basicConfig(format = '%(asctime)s : %(levelname)s : %(message)s',
                    level = logging.INFO)

In [41]:
num_features = 300    # Word vector dimensionality                      
min_word_count = 40   # Minimum word count                        
num_workers = 4       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

### Initialize and train the model.

This will take some time: with `num_workers = 4` 
```
CPU times: user 8min 20s, sys: 13.3 s, total: 8min 33s
```

In [43]:
%%time
model = word2vec.Word2Vec(sentences, 
                          workers = num_workers, 
                          size = num_features, 
                          min_count = min_word_count,
                          window = context, 
                          sample = downsampling)

2016-11-10 22:19:00,277 : INFO : collecting all words and their counts
2016-11-10 22:19:00,281 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2016-11-10 22:19:00,431 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2016-11-10 22:19:00,531 : INFO : PROGRESS: at sentence #20000, processed 451867 words, keeping 24947 word types
2016-11-10 22:19:00,629 : INFO : PROGRESS: at sentence #30000, processed 671290 words, keeping 30033 word types
2016-11-10 22:19:00,727 : INFO : PROGRESS: at sentence #40000, processed 897790 words, keeping 34347 word types
2016-11-10 22:19:00,822 : INFO : PROGRESS: at sentence #50000, processed 1116929 words, keeping 37760 word types
2016-11-10 22:19:00,919 : INFO : PROGRESS: at sentence #60000, processed 1338370 words, keeping 40722 word types
2016-11-10 22:19:01,019 : INFO : PROGRESS: at sentence #70000, processed 1561505 words, keeping 43332 word types
2016-11-10 22:19:01,116 : INFO : PROGRESS: 

CPU times: user 8min 20s, sys: 13.3 s, total: 8min 33s
Wall time: 2min 46s


If you don't plan to train the model any further, calling `init_sims` will make the model much more memory-efficient.

In [44]:
model.init_sims(replace = True)

2016-11-10 22:22:11,225 : INFO : precomputing L2-norms of word weight vectors


It can be helpful to create a meaningful model name and save the model for later use. You can load it later using `Word2Vec.load()`

In [45]:
model_name = "300features_40minwords_10context"
model.save(model_name)

2016-11-10 22:22:13,391 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2016-11-10 22:22:13,444 : INFO : not storing attribute syn0norm
2016-11-10 22:22:13,446 : INFO : not storing attribute cum_table


### Exploring the Model Results

In [46]:
model.doesnt_match("man woman child kitchen".split())

'kitchen'

In [47]:
model.doesnt_match("france england germany berlin".split())

'berlin'

In [48]:
model.doesnt_match("paris berlin london austria".split())

'paris'

In [49]:
model.most_similar("man")

[('woman', 0.6106343865394592),
 ('monk', 0.5867377519607544),
 ('lady', 0.585590124130249),
 ('lad', 0.573781430721283),
 ('soldier', 0.5376678705215454),
 ('businessman', 0.5143798589706421),
 ('farmer', 0.5093753337860107),
 ('men', 0.5063858032226562),
 ('guy', 0.5043983459472656),
 ('person', 0.5014356970787048)]

In [50]:
 model.most_similar("queen")

[('bride', 0.6558289527893066),
 ('princess', 0.6477103233337402),
 ('latifah', 0.6186422109603882),
 ('goddess', 0.6099314093589783),
 ('stepmother', 0.5945958495140076),
 ('eva', 0.5876355767250061),
 ('prince', 0.5868668556213379),
 ('victoria', 0.580491840839386),
 ('mistress', 0.5729061365127563),
 ('angela', 0.570813775062561)]

In [51]:
model.most_similar("awful")

[('terrible', 0.7688319683074951),
 ('atrocious', 0.7546607255935669),
 ('abysmal', 0.7259805798530579),
 ('horrible', 0.7223121523857117),
 ('dreadful', 0.7169843912124634),
 ('horrendous', 0.7052638530731201),
 ('horrid', 0.6820660829544067),
 ('lousy', 0.6667404770851135),
 ('appalling', 0.6601595878601074),
 ('amateurish', 0.6331586837768555)]

-----
# Part 3: More Fun With Word Vectors

[source](https://www.kaggle.com/c/word2vec-nlp-tutorial/details/part-3-more-fun-with-word-vectors)

## Numeric Representations of Words

The Word2Vec model trained in Part 2 consists of a feature vector for each word in the vocabulary, stored in a `numpy` array called "`syn0`":

In [58]:
model = word2vec.Word2Vec.load("300features_40minwords_10context")

2016-11-10 22:38:41,912 : INFO : loading Word2Vec object from 300features_40minwords_10context
2016-11-10 22:38:42,781 : INFO : setting ignored attribute syn0norm to None
2016-11-10 22:38:42,783 : INFO : setting ignored attribute cum_table to None


In [59]:
type(model.syn0)

numpy.ndarray

In [60]:
model.syn0.shape

(16490, 300)

The number of rows in `syn0` is the number of words in the model's vocabulary, and the number of columns corresponds to the size of the feature vector, which we set in Part 2.  Setting the minimum word count to 40 gave us a total vocabulary of 16,492 words with 300 features apiece. Individual word vectors can be accessed in the following way:

In [62]:
# model["flower"]

-----
## From Words To Paragraphs, Attempt 1: Vector Averaging

In [107]:
def makeFeatureVec(words, model, num_features):
    """Function to average all of the word vectors in a given
    paragraph
    """
    
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,), dtype = "float64")
    nwords = 0.

    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)

    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    """Given a set of reviews (each one a list of words), 
    calculate the average feature vector 
    for each one and return a 2D numpy array 
    """
    # Initialize a counter
    counter = 0.
    
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features), dtype = "float64")
    
    # Loop through the reviews
    for review in reviews:
        # Print a status message every 1000th review
        if counter % 1000. == 0.:
            ("Review %d of %d" % (counter, len(reviews)))
        
        # Call the function (defined above) that makes average feature vectors
        reviewFeatureVecs[counter] = makeFeatureVec(review, model, num_features)
        # Increment the counter
        counter = counter + 1.
        
    return reviewFeatureVecs

Calculate average feature vectors for training and testing sets,
using the functions we defined above. Notice that we now use stop word
removal.

In [108]:
%%time
clean_train_reviews = []

for review in train["review"]:
    clean_train_reviews.append(review_to_wordlist(review, 
                                                  remove_stopwords = True))

trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

CPU times: user 1min 16s, sys: 2.86 s, total: 1min 19s
Wall time: 1min 21s


In [109]:
%%time
clean_test_reviews = []

for review in test["review"]:
    clean_test_reviews.append(review_to_wordlist(review, remove_stopwords=True))

testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

CPU times: user 1min 16s, sys: 10.4 s, total: 1min 27s
Wall time: 1min 34s


### Random Forest
Next, use the average paragraph vectors to train a random forest. Note that, as in Part 1, we can only use the labeled training reviews to train the model. 

In [110]:
forest = RandomForestClassifier(n_estimators = 100)

In [ ]:
%%time
# forest = forest.fit(trainDataVecs, train["sentiment"])

I get an error here:
```
ValueError: Input contains NaN, infinity or a value too large for dtype('float32').
```

In [ ]:
%%time
# result = forest.predict(testDataVecs)

In [ ]:
# # Write the test results 
# output = pd.DataFrame(data = {"id":test["id"], "sentiment":result})
# output.to_csv("Word2Vec_AverageVectors.csv", 
#               index = False, 
#               quoting = 3)

-----
## From Words to Paragraphs, Attempt 2: Clustering 

Word2Vec creates clusters of semantically related words, so another possible approach is to exploit the similarity of words within a cluster. Grouping vectors in this way is known as "vector quantization." To accomplish this, we first need to find the centers of the word clusters, which we can do by using a clustering algorithm such as K-Means.

Set "k" (`num_clusters`) to be 1/5th of the vocabulary size, or an average of 5 words per cluster

In [79]:
word_vectors = model.syn0
num_clusters = word_vectors.shape[0] // 5

In [80]:
kmeans_clustering = KMeans(n_clusters = num_clusters)

In [81]:
%%time
idx = kmeans_clustering.fit_predict(word_vectors)

CPU times: user 14min 57s, sys: 49.4 s, total: 15min 46s
Wall time: 11min 9s


Create a Word / Index dictionary, mapping each vocabulary word to a cluster number                                                                                            

In [82]:
word_centroid_map = dict(zip( model.index2word, idx ))

This is a little abstract, so let's take a closer look at what our clusters contain. Your clusters may differ, as Word2Vec relies on a random number seed. Here is a loop that prints out the words for clusters 0 through 9:



In [87]:
for cluster in range(10):
    print("\nCluster %d" % cluster)

    # Find all of the words for that cluster number, and print them out
    words = []
    for i in range(len(word_centroid_map.values())):
        if( list(word_centroid_map.values())[i] == cluster ):
            words.append(list(word_centroid_map.keys())[i])
    print(words)


Cluster 0
['pumpkinhead', 'sematary']

Cluster 1
['lobby', 'storage']

Cluster 2
['mirren', 'lansbury', 'walters', 'helen', 'ellen', 'chloe']

Cluster 3
['individual', 'external']

Cluster 4
['blaine', 'halle', 'durante']

Cluster 5
['rogue', 'drunken', 'hapless', 'heavies', 'bumbling', 'misfit']

Cluster 6
['avoids']

Cluster 7
['charge', 'hostage', 'retirement', 'custody', 'temporarily', 'willingly']

Cluster 8
['records', 'stones', 'notes', 'albums', 'videos', 'recordings']

Cluster 9
['emperor', 'inland', 'empire']


Now we have a cluster (or "centroid") assignment for each word, and we can define a function to convert reviews into bags-of-centroids. This works just like Bag of Words but uses semantically related clusters instead of individual words.

* The number of clusters is equal to the highest cluster index in the word / centroid map

In [88]:
def create_bag_of_centroids(wordlist, word_centroid_map):
    num_centroids = max(word_centroid_map.values()) + 1
    
    # Pre-allocate the bag of centroids vector (for speed)
    bag_of_centroids = np.zeros( num_centroids, dtype="float32" )
    
    # Loop over the words in the review. If the word is in the vocabulary,
    # find which cluster it belongs to, and increment that cluster count 
    # by one
    for word in wordlist:
        if word in word_centroid_map:
            index = word_centroid_map[word]
            bag_of_centroids[index] += 1
    
    return bag_of_centroids

Transform the training set reviews into bags of centroids

In [91]:
%%time
train_centroids = np.zeros((train["review"].size, num_clusters),
                           dtype = "float32")

counter = 0

for review in clean_train_reviews:
    train_centroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1

CPU times: user 43.2 s, sys: 1.59 s, total: 44.8 s
Wall time: 47.4 s


Repeat for test reviews 

In [93]:
%%time
test_centroids = np.zeros((test["review"].size, num_clusters), 
                          dtype = "float32")

counter = 0

for review in clean_test_reviews:
    test_centroids[counter] = create_bag_of_centroids(review, word_centroid_map)
    counter += 1

CPU times: user 40.1 s, sys: 888 ms, total: 41 s
Wall time: 41.5 s


### Random Forest

Fit a random forest and extract predictions 

In [95]:
forest = RandomForestClassifier(n_estimators = 100)

In [96]:
%%time
forest = forest.fit(train_centroids, train["sentiment"])

CPU times: user 1min 11s, sys: 1.13 s, total: 1min 12s
Wall time: 1min 14s


In [97]:
%%time
result = forest.predict(test_centroids)

CPU times: user 1.93 s, sys: 431 ms, total: 2.36 s
Wall time: 3.38 s


In [98]:
# # Write the test results 
# output = pd.DataFrame(data={"id":test["id"], "sentiment":result})
# output.to_csv("BagOfCentroids.csv", 
#               index = False, 
#               quoting = 3)